In [ ]:
import sys
import os
import pandas as pd

### IMPORTANT ###
# Make sure you are correctly appending the path
# Otherwise the imports will not work!
sys.path.append("/Users/pvs262/Documents/rec-sys-dynamics/code")
from src.analysis.cluster import movielens, cluster

In [ ]:
# Import ML100K Lenskit Dataset
from lenskit.datasets import ML100K
dataset = ML100K('/Users/pvs262/Library/Python/3.8/lib/python/site-packages/lenskit/data/ml-100k')
ratings = dataset.ratings
movies = dataset.movies

In [ ]:
# Check loaded ratings dataset
ratings.head()

In [ ]:
# Check loaded movies dataset
movies.head()

In [ ]:
# Instantiate MovieLens object
data = movielens(movies, ratings)
# Get User-Item Interaction Matrix from MovieLens object
dataUI = data.UserItem()  

In [ ]:
# Let's print the original User-Item Interaction Matrix
dataUI

In [ ]:
# Instantiate Clustering object
ML_clusters = cluster(dataUI)

In [ ]:
%%time
# One-time set up to choose the number of latent features we want. 
# If you want to change the number of latent features in the reduced dataset, you need to use cluster.SVD(n) again.
ML_clusters.svd(3)

In [ ]:
%%time
# Function to add user/item with ID
ML_clusters.add_user(len(dataUI)+1) 
ML_clusters.add_item(len(dataUI.columns)+1)

# Function to add user/item automatically (it returns the ID of the new user/item)
ML_clusters.add_user_auto() 
ML_clusters.add_item_auto()

# Function to add ratings for EXISTING users and items where cluster.add_interactions(user_id, item_id, rating)
ML_clusters.add_interactions([4],[2],[3])

In [ ]:
# Let's print the new User-Item Interaction Matrix
dataUI

In [ ]:
%%time
### IMPORTANT: Make sure you train the model/clusters before plotting them using cluster.gmm(n, covariance_type, df)

# cluster.gmm(n, covariance_type, df) calculates SVD to find updated reduced dataset and then clusters
# returns cluster number for each user and the probability of belonging to each of the n clusters

[latent, probas] = ML_clusters.gmm(n=3,covariance_type="full",df='all')

In [ ]:
probas

In [ ]:
latent

In [ ]:
probas['sum'] = probas['proba_C0']+probas['proba_C1']+probas['proba_C2']
probas['proba_C0'].sum()

In [ ]:
probas['proba_C1'].sum()

In [ ]:
probas['proba_C2'].sum()

In [ ]:
# plot data and clusters (True for colour-coded clusters, if False it will just plot latent features in monotone)
# Thi functions currently only work in JupyterNotebook.                            
ML_clusters.plot_scatter(True, 'gmm')  

In [ ]:
sys.path.append("/Users/pvs262/Documents/rec-sys-dynamics/code")
import src.Datasets.movielens_recreate as dataset
from src.algorithm.cosin import CosinSimilarity

In [ ]:
ratings = pd.read_parquet(os.path.dirname(dataset.__file__)+'/ratings.parquet.gzip')

In [ ]:
ratings

In [ ]:
algo = CosinSimilarity()
algo.fit(ratings)
UI = algo.rating_matrix_
UI